In [172]:
import pandas as pd
import numpy as np
import scipy.constants as const
import handcalcs.render
import matplotlib.pyplot as plt
import scipy.optimize as opt
import math

# Расчетная часть

In [113]:
%%render
f_0  = 9.18*const.giga
lambda_0 = const.c/f_0 #в метрах
lambda_0 = round(lambda_0/const.milli,1) #в миллиметрах

<IPython.core.display.Latex object>

## Рупорная и рупорная с корректирующей линзой антенны

In [205]:
%%render
A = B = 50 #мм

<IPython.core.display.Latex object>

##### Ширина главного лепестка ДН в Е-плоскости

In [210]:
%%render
Theta_0_E = 2*lambda_0/B
Theta_05_E = 0.88*lambda_0/B

<IPython.core.display.Latex object>

In [211]:
round(math.degrees(Theta_0_E),2),round(math.degrees(Theta_05_E),2)

(74.94, 32.97)

##### Ширина главного лепестка ДН в H-плоскости

In [208]:
%%render
Theta_0_H = 3*lambda_0/A
Theta_05_H = 1.22*lambda_0/A

<IPython.core.display.Latex object>

In [212]:
round(math.degrees(Theta_0_H),2),round(math.degrees(Theta_05_H),2)

(112.41, 45.72)

##### Эффективная площадь

In [217]:
%%render
upsilon1 = upsilon2 = 0.78
upsilon = upsilon1*upsilon2
S_p = A*B
S_ef = S_p*upsilon

<IPython.core.display.Latex object>

##### КНД

In [219]:
%%render
D_0 = 4*const.pi*S_ef/lambda_0**2

<IPython.core.display.Latex object>

##### КПД

In [223]:
%%render
eta = 1 #принимают для удобства расчетов
#мб здесь что-то не так

<IPython.core.display.Latex object>

##### Коэффициент усиления

In [224]:
%%render
G_0 = D_0*eta

<IPython.core.display.Latex object>

## Диэлектрическая стержневая антенна

In [140]:
L = 15 #в сантиметрах
L = 150  #в миллиметрах 
Pavg = 1.1 #в методе

In [166]:
u_z = lambda Theta: const.pi*L/lambda_0*(Pavg - np.cos(Theta))
F_E = lambda Theta: np.cos(Theta)*np.sin(u_z(Theta))/u_z(Theta)
F_H = lambda Theta: np.sin(u_z(Theta))/u_z(Theta)

In [181]:
#Функция расчета ДН по уровню 0 и 0.5 и перевод в градусы
def Find_width_radiation_pattern(func):
    Theta_0 = round(opt.fsolve(func,[0.9])[0],2)
    Theta_05 = round(func(0)/np.sqrt(2),2)
    return Theta_0,round(math.degrees(Theta_0),2),Theta_05,round(math.degrees(Theta_05),2)

##### Ширина главного лепестка ДН в Е-плоскости

In [176]:
Find_width_radiation_pattern(F_E)

(0.84, 48.13, 0.49, 28.07)

In [167]:
Theta_0_E = round(opt.fsolve(F_E,[0.9])[0],2)
#Решаю численно уравнение F_E(Theta_0) = 0
Theta_0_E

0.84

In [168]:
Theta_05_E = round(F_E(0)/np.sqrt(2),2)
Theta_05_E

0.49

##### Ширина главного лепестка ДН в H-плоскости

In [180]:
Find_width_radiation_pattern(F_H)

(0.84, 48.13, 0.49, 28.07)

In [169]:
Theta_0_H = round(opt.fsolve(F_H,[0.9])[0],2)
#Решаю численно уравнение F_E(Theta_0) = 0
Theta_0_H

0.84

In [170]:
Theta_05_H = round(F_E(0)/np.sqrt(2),2)
Theta_05_H

0.49

##### КНД

In [184]:
%%render
k_D = 5 #лежит в интервале [4..6]
D_0 = k_D*L/lambda_0 #мм/мм

<IPython.core.display.Latex object>

##### КПД

In [190]:
%%render
K_alpha = 0.305
tan_delta = 1e-3
epsilon_r = 2.56
alpha = 27.29*epsilon_r*tan_delta*K_alpha/lambda_0
eta = math.exp(-2*alpha*L)

<IPython.core.display.Latex object>

##### Коэффициент усиления и эффективная площадь

In [192]:
%%render
G_0 = eta*D_0
S_ef = 1/(4*const.pi)*G_0*lambda_0**2 #мм^2

<IPython.core.display.Latex object>

# Экспериментальная часть

## 1. Рупорная антенна

In [114]:
%%render
KSW = np.sqrt(6.51/5.2) #КСВН

<IPython.core.display.Latex object>

In [115]:
#Рупорная антенна
data = {
    "Расстояние, на которое экран перемещен,мм":
              [160,163,166,169,173,176,179,182,186,189],
    "Umax,мВ":[6.7,7.1,7.8,7.6,6.8,6.6,7.2,7.3,6.8,6.8],
    "Umin,мВ":[4.8,4.2,4.1,4.2,4.8,4.8,4.2,4.2,4.8,4.6]
       }
horn_antenna = pd.DataFrame(data)
horn_antenna

,"Расстояние, на которое экран перемещен,мм","Umax,мВ","Umin,мВ"
0,160,6.7,4.8
1,163,7.1,4.2
2,166,7.8,4.1
3,169,7.6,4.2
4,173,6.8,4.8
5,176,6.6,4.8
6,179,7.2,4.2
7,182,7.3,4.2
8,186,6.8,4.8
9,189,6.8,4.6


In [116]:
horn_antenna["КСВН"] = round(np.sqrt(horn_antenna["Umax,мВ"]/horn_antenna["Umin,мВ"]),2)
#мВ/мВ
horn_antenna["G_oi"] = 4*np.pi*2*horn_antenna["Расстояние, на которое экран перемещен,мм"]/lambda_0 
#оставляем миллиметры
horn_antenna["G_oi"] = round(horn_antenna["G_oi"]*(horn_antenna["КСВН"] - 1)/(horn_antenna["КСВН"] + 1),2)
#возможна ошибка в формуле
horn_antenna

,"Расстояние, на которое экран перемещен,мм","Umax,мВ","Umin,мВ",КСВН,G_oi
0,160,6.7,4.8,1.18,10.15
1,163,7.1,4.2,1.30,16.34
2,166,7.8,4.1,1.38,20.37
3,169,7.6,4.2,1.35,19.35
4,173,6.8,4.8,1.19,11.54
5,176,6.6,4.8,1.17,10.60
6,179,7.2,4.2,1.31,18.46
7,182,7.3,4.2,1.32,19.29
8,186,6.8,4.8,1.19,12.40
9,189,6.8,4.6,1.22,14.40


In [117]:
#round(sum(horn_antenna["G_oi"])/len(horn_antenna["G_oi"]),2)
G_0 = round(horn_antenna["G_oi"].mean(),2)
G_0

15.29

In [118]:
round(10*np.log10(G_0),2)# в дБ

11.84

In [119]:
#S эффективная в мм^2
S_ef = 1/(4*np.pi)*G_0*(lambda_0)**2
round(S_ef,2)

1301.05

## 2. Рупорная антенна с корректирующей линзой

In [120]:
%%render
KSW= np.sqrt(2.2/2) #КСВН

<IPython.core.display.Latex object>

In [121]:
data = {
    "Расстояние, на которое экран перемещен,мм":
              [305.8,309,312.3,315.6,318.8,322.1,325.4,328.7,332,325.2],
    "Umax,мВ":[2.2  ,2.6,2.58 ,2.4  ,2.55 ,2.6  ,2.58 ,2.58 ,2.4,2.36 ],
    "Umin,мВ":[1.4  ,1.7,1.79 ,1.8  ,1.9  ,1.8  ,1.7  ,1.8  ,1.9,2    ]
       }
horn_antenna_with_corrective_lens = pd.DataFrame(data)
horn_antenna_with_corrective_lens

,"Расстояние, на которое экран перемещен,мм","Umax,мВ","Umin,мВ"
0,305.8,2.20,1.40
1,309.0,2.60,1.70
2,312.3,2.58,1.79
3,315.6,2.40,1.80
4,318.8,2.55,1.90
5,322.1,2.60,1.80
6,325.4,2.58,1.70
7,328.7,2.58,1.80
8,332.0,2.40,1.90
9,325.2,2.36,2.00


In [122]:
horn_antenna_with_corrective_lens["КСВН"] = round(np.sqrt(horn_antenna_with_corrective_lens["Umax,мВ"]/horn_antenna_with_corrective_lens["Umin,мВ"]),2)
#мВ/мВ
horn_antenna_with_corrective_lens["G_oi"] = 4*np.pi*2*horn_antenna_with_corrective_lens["Расстояние, на которое экран перемещен,мм"]/lambda_0 
#оставляем миллиметры
horn_antenna_with_corrective_lens["G_oi"] = round(horn_antenna_with_corrective_lens["G_oi"]*(horn_antenna_with_corrective_lens["КСВН"] - 1)/(horn_antenna_with_corrective_lens["КСВН"] + 1),2)
#возможна ошибка в формуле
horn_antenna_with_corrective_lens

,"Расстояние, на которое экран перемещен,мм","Umax,мВ","Umin,мВ",КСВН,G_oi
0,305.8,2.20,1.40,1.25,26.11
1,309.0,2.60,1.70,1.24,25.45
2,312.3,2.58,1.79,1.20,21.82
3,315.6,2.40,1.80,1.15,16.92
4,318.8,2.55,1.90,1.16,18.15
5,322.1,2.60,1.80,1.20,22.51
6,325.4,2.58,1.70,1.23,25.79
7,328.7,2.58,1.80,1.20,22.97
8,332.0,2.40,1.90,1.12,14.44
9,325.2,2.36,2.00,1.09,10.76


In [123]:
#round(sum(horn_antenna["G_oi"])/len(horn_antenna["G_oi"]),2)
G_0 = round(horn_antenna_with_corrective_lens["G_oi"].mean(),2)
G_0

20.49

In [124]:
round(10*np.log10(G_0),2)# в дБ

13.12

In [125]:
#S эффективная в мм^2
S_ef = 1/(4*np.pi)*G_0*(lambda_0)**2
round(S_ef,2)

1743.52

## 3. Диэлектрическая стержневая антенна

In [126]:
%%render
KSW= np.sqrt(2/1.3) #КСВН

<IPython.core.display.Latex object>

In [127]:
data = {
    "Расстояние, на которое экран перемещен,мм":
              [230,233.8,236.6,239.9,243.2,246.5,249.8,253.1,256.4,259.7],
    "Umax,мВ":[2.9,2.5  ,2.2  ,2.5  ,2.9  ,3    ,2.7  ,2.2  ,2.2  ,2.7  ],
    "Umin,мВ":[0.8,1.1  ,1.4  ,1.1  ,0.8  ,0.8  ,0.9  ,1.3  ,1.3  ,0.9  ]
       }
dielectric_antenna = pd.DataFrame(data)
dielectric_antenna

,"Расстояние, на которое экран перемещен,мм","Umax,мВ","Umin,мВ"
0,230.0,2.9,0.8
1,233.8,2.5,1.1
2,236.6,2.2,1.4
3,239.9,2.5,1.1
4,243.2,2.9,0.8
5,246.5,3.0,0.8
6,249.8,2.7,0.9
7,253.1,2.2,1.3
8,256.4,2.2,1.3
9,259.7,2.7,0.9


In [128]:
dielectric_antenna["КСВН"] = round(np.sqrt(dielectric_antenna["Umax,мВ"]/dielectric_antenna["Umin,мВ"]),2)
#мВ/мВ
dielectric_antenna["G_oi"] = 4*np.pi*2*dielectric_antenna["Расстояние, на которое экран перемещен,мм"]/lambda_0 
#оставляем миллиметры
dielectric_antenna["G_oi"] = round(dielectric_antenna["G_oi"]*(dielectric_antenna["КСВН"] - 1)/(dielectric_antenna["КСВН"] + 1),2)
#возможна ошибка в формуле
dielectric_antenna

,"Расстояние, на которое экран перемещен,мм","Umax,мВ","Umin,мВ",КСВН,G_oi
0,230.0,2.9,0.8,1.90,54.86
1,233.8,2.5,1.1,1.51,36.51
2,236.6,2.2,1.4,1.25,20.21
3,239.9,2.5,1.1,1.51,37.46
4,243.2,2.9,0.8,1.90,58.01
5,246.5,3.0,0.8,1.94,60.57
6,249.8,2.7,0.9,1.73,51.34
7,253.1,2.2,1.3,1.30,25.37
8,256.4,2.2,1.3,1.30,25.70
9,259.7,2.7,0.9,1.73,53.37


##### Коэффициент усиления и эффективная площадь

In [129]:
#round(sum(horn_antenna["G_oi"])/len(horn_antenna["G_oi"]),2)
G_0 = round(dielectric_antenna["G_oi"].mean(),2)
G_0

42.34

In [130]:
round(10*np.log10(G_0),2)# в дБ

16.27

In [131]:
#S эффективная в мм^2
S_ef = 1/(4*np.pi)*G_0*(lambda_0)**2
round(S_ef,2)

3602.77